In [1]:
import os
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, average_precision_score
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 配置参数
class Config:
    # 路径配置
    DATA_ROOT = "D:/Code_pytorch/xianyu/hand detection"
    TRAIN_IMAGES = os.path.join(DATA_ROOT, "training_dataset/training_dataset/training_data/images")
    TRAIN_LABELS = os.path.join(DATA_ROOT, "labels_fast_rcnn")
    TEST_IMAGES = os.path.join(DATA_ROOT, "training_dataset/training_dataset/training_data/images")
    TEST_LABELS = os.path.join(DATA_ROOT, "labels_fast_rcnn")
    OUTPUT_DIR = os.path.join(DATA_ROOT, "output")
    MODEL_SAVE_PATH = os.path.join(OUTPUT_DIR, "faster_rcnn_model.pth")
    
    # 训练参数
    NUM_CLASSES = 2  # 背景 + 目标类别数
    BATCH_SIZE = 10
    NUM_EPOCHS = 20
    LEARNING_RATE = 0.005
    MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0005
    
    # 设备配置
    DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
# 自定义数据集类
class HandDataset(Dataset):
    def __init__(self, image_dir, label_dir, train=False, transforms=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.train = train
        self.image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # 加载图像
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        original_width, original_height = image.size
        
        # 调整图像大小到112x112
        new_size = (112, 112)
        image = F.resize(image, new_size)
        
        # 计算缩放比例
        width_scale = new_size[0] / original_width
        height_scale = new_size[1] / original_height
        
        # 加载标签
        label_path = os.path.join(self.label_dir, os.path.splitext(img_name)[0] + ".txt")
        boxes = []
        labels = []
        
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                
                class_id = int(parts[0])
                x_min = float(parts[1])
                y_min = float(parts[2])
                x_max = float(parts[3])
                y_max = float(parts[4])
                
                # 缩放坐标
                x_min = x_min * width_scale
                y_min = y_min * height_scale
                x_max = x_max * width_scale
                y_max = y_max * height_scale
                
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(class_id + 1)  # 0保留给背景

        # 转换为张量
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        # 数据增强：随机水平翻转
        if self.train and random.random() < 0.5:
            image = F.hflip(image)
            boxes[:, [0, 2]] = 112 - boxes[:, [2, 0]]

        image = F.to_tensor(image)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx]),
            "area": (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]),
            "iscrowd": torch.zeros((len(boxes),), dtype=torch.int64)
        }

        if self.transforms:
            image = self.transforms(image)

        return image, target

In [4]:
# 创建模型
def create_model(num_classes):
    backbone = torchvision.models.resnet50(pretrained=True)
    backbone = torch.nn.Sequential(*list(backbone.children())[:-2])
    backbone.out_channels = 2048
    

    anchor_generator = AnchorGenerator(
        sizes=((16, 32, 64, 128, 256),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )  
    
    
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )
    
    model = FasterRCNN(
        backbone,
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler
    )
    
    return model

In [5]:
# IoU计算函数
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

In [6]:
# 训练函数（添加损失记录）
def train_model(model, train_loader, optimizer, lr_scheduler, num_epochs):
    model.train()
    train_losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for images, targets in train_loader:
            images = list(image.to(Config.DEVICE) for image in images)
            targets = [{k: v.to(Config.DEVICE) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            
            epoch_loss += losses.item()
        
        epoch_loss /= len(train_loader)
        train_losses.append(epoch_loss)
        lr_scheduler.step()
        print(f"Epoch {epoch+1}/{num_epochs} Loss: {epoch_loss:.4f}")
    return train_losses

In [7]:
# 评估函数（改进指标计算）
def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for images, targets in test_loader:
            images = list(img.to(Config.DEVICE) for img in images)
            outputs = model(images)
            
            for i, output in enumerate(outputs):
                pred_boxes = output['boxes'].cpu().numpy()
                pred_labels = output['labels'].cpu().numpy()
                pred_scores = output['scores'].cpu().numpy()
                target_boxes = targets[i]['boxes'].cpu().numpy()
                target_labels = targets[i]['labels'].cpu().numpy()
                
                all_preds.append((pred_boxes, pred_labels, pred_scores))
                all_targets.append((target_boxes, target_labels))
    
    # 计算指标
    precision, recall, ap = calculate_metrics(all_preds, all_targets)
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, AP: {ap:.4f}")
    return precision, recall, ap

In [8]:
def calculate_metrics(preds, targets, iou_threshold=0.5):
    scores_list = []
    tp_list = []
    fp_list = []
    total_gt = 0
    
    for (pred_boxes, pred_labels, pred_scores), (gt_boxes, gt_labels) in zip(preds, targets):
        # 只处理类别为1的预测
        valid_mask = pred_labels == 1
        pred_boxes = pred_boxes[valid_mask]
        pred_scores = pred_scores[valid_mask]
        
        # 按置信度排序
        sorted_indices = np.argsort(-pred_scores)
        pred_boxes = pred_boxes[sorted_indices]
        pred_scores = pred_scores[sorted_indices]
        
        # 初始化匹配状态
        gt_matched = np.zeros(len(gt_boxes), dtype=bool)
        tp = np.zeros(len(pred_boxes), dtype=int)
        fp = np.zeros(len(pred_boxes), dtype=int)
        
        for i, pred_box in enumerate(pred_boxes):
            best_iou = 0.0
            best_gt = -1
            
            for j, gt_box in enumerate(gt_boxes):
                if gt_matched[j]:
                    continue
                iou = calculate_iou(pred_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_gt = j
            
            if best_iou >= iou_threshold and best_gt != -1:
                tp[i] = 1
                gt_matched[best_gt] = True
            else:
                fp[i] = 1
        
        scores_list.extend(pred_scores)
        tp_list.extend(tp)
        fp_list.extend(fp)
        total_gt += len(gt_boxes)
    
    # 转换为数组并排序
    scores = np.array(scores_list)
    tp = np.array(tp_list)
    fp = np.array(fp_list)
    
    indices = np.argsort(-scores)
    tp = tp[indices]
    fp = fp[indices]
    
    # 计算累积指标
    cum_tp = np.cumsum(tp)
    cum_fp = np.cumsum(fp)
    
    precision = cum_tp / (cum_tp + cum_fp + 1e-6)
    recall = cum_tp / (total_gt + 1e-6)
    
    # 计算MAP（PASCAL VOC风格）
    ap = 0.0
    for t in np.arange(0., 1.1, 0.1):
        mask = recall >= t
        if np.any(mask):
            ap += np.max(precision[mask]) / 11
    
    # 最终precision和recall
    final_precision = cum_tp[-1]/(cum_tp[-1]+cum_fp[-1]) if (cum_tp[-1]+cum_fp[-1])>0 else 0
    final_recall = cum_tp[-1]/total_gt if total_gt>0 else 0
    
    return final_precision, final_recall, ap

In [9]:
# 可视化预测结果
def visualize_predictions(model, test_loader, num_images=3):
    model.eval()
    images, targets = next(iter(test_loader))
    
    with torch.no_grad():
        images = list(img.to(Config.DEVICE) for img in images)
        outputs = model(images)
    
    for i in range(min(num_images, len(images))):
        img = images[i].cpu().permute(1, 2, 0).numpy()
        img = (img * 255).astype(np.uint8)
        img = Image.fromarray(img)
        
        draw = ImageDraw.Draw(img)
        
        # 真实框（绿色）
        for box in targets[i]['boxes']:
            draw.rectangle(box.tolist(), outline=(0, 255, 0), width=2)
        
        # 预测框（红色）
        for box in outputs[i]['boxes'].cpu().numpy():
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=2)
        
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Predictions (Red) vs Ground Truth (Green)")
        plt.show()

In [10]:
# 绘制损失曲线
def plot_loss_curve(train_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.legend()
    plt.savefig(os.path.join(Config.OUTPUT_DIR, 'loss_curve.png'))
    plt.show()

In [ ]:
# 主程序
def main():
    os.makedirs(Config.OUTPUT_DIR, exist_ok=True)
    
    # 准备数据集
    train_dataset = HandDataset(
        Config.TRAIN_IMAGES,
        Config.TRAIN_LABELS,
        train=True
    )
    
    test_dataset = HandDataset(
        Config.TEST_IMAGES,
        Config.TEST_LABELS
    )
    
    # 创建数据加载器
    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=True,
        collate_fn=lambda x: tuple(zip(*x)))
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=False,
        collate_fn=lambda x: tuple(zip(*x)))
    
    # 初始化模型
    model = create_model(Config.NUM_CLASSES)
    model.to(Config.DEVICE)
    
    # 定义优化器
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(
        params,
        lr=Config.LEARNING_RATE,
        momentum=Config.MOMENTUM,
        weight_decay=Config.WEIGHT_DECAY
    )
    
    # 学习率调度器
    lr_scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=5,
        gamma=0.1
    )
    
    # 训练模型
    print("开始训练...")
    train_losses = train_model(model, train_loader, optimizer, lr_scheduler, Config.NUM_EPOCHS)
    
    # 绘制损失曲线
    plot_loss_curve(train_losses)
    
    # 保存模型
    torch.save(model.state_dict(), Config.MODEL_SAVE_PATH)
    print(f"模型已保存到 {Config.MODEL_SAVE_PATH}")
    
    # 评估模型
    print("\n正在评估测试集...")
    precision, recall, ap = evaluate(model, test_loader)
    print(f"最终评估结果：\nPrecision: {precision:.4f}, Recall: {recall:.4f}, AP: {ap:.4f}")
    
    # 可视化预测
    print("\n正在生成预测可视化...")
    visualize_predictions(model, test_loader)

if __name__ == "__main__":
    main()